## Preparation

In [1]:
import numpy as np
from sympy import *
# from sympy.solvers import solve
import warnings
from scipy.linalg import solve_continuous_lyapunov
from sklearn import linear_model
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning) 
init_printing(use_unicode=True)

In [2]:
def simulate_M(seed: int, p: int) -> Array:
    np.random.seed(seed=seed)
    normal_matrix = np.random.normal(0, 1, (p, p))
    for i in range(p): # adjust diagonal entries s.t. M stable
        normal_matrix[i, i] = - np.sum(np.abs(normal_matrix[i, :]))
    return normal_matrix

def num_sym(p: int) -> int:
    return int(p * (p + 1) / 2)

def create_A_Sigma(cov: Array, p: int) -> Array:
    row_num = num_sym(p=p)
    col_num = int(p * p)
    A_Sigma = np.empty(shape=(row_num, col_num), dtype=float)
    for l in range(p):
        for k in range(l+1):
            for i in range(p):
                for j in range(p):
                    if (j != k) & (j != l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = 0
                    elif (j == k) & (k != l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = cov[l, i]
                    elif (j == l) & (l != k):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = cov[k, i]
                    elif (j == k) & (j == l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = 2 * cov[j, i]
    return A_Sigma

def create_A(A_Sigma: Array, mean: Array, p: int) -> Array:
    identity = mean[0] * eye(p)
    for i in range(1, p):
        identity = np.hstack((identity, mean[i] * eye(p)))
    return np.vstack((A_Sigma, identity))

In [3]:
num_exp = 1000 # number of true matrices per case
num_sim = 1000 # number of simulations per true matrix

index_intervention = 0
b = 2

Universally:
* save estimates and their Frobenius distances to the true $M^*$
* $\texttt{frob = -1}$ if no result could be produced

## 2 nodes

In [4]:
p = 2

C = 2 * np.eye(p)
vech_C = np.array([C[i, j] for i in range(p) for j in range(i, p)])
unit_v = np.eye(p)[:, index_intervention]

reg_param = [0.001, 0.01, 0.1, 1]
num_param = len(reg_param) + 1 # also linear regression

### $\texttt{index\_deletion = 0}$
***Results:*** 
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{2.311}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{2.716}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{2.546}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.939}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.715}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.64}$.

In [5]:
'''
index_deletion = 0
offset = 1

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, 4 + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_0[seed-offset, 0] = est_M
    frobnorm_2_0[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_0[seed-offset, index+1] = est_M
        frobnorm_2_0[seed-offset, index+1] = frob
        

    # Ordinary solution with sympy.solve
    solution = solve(emp_A * vec_M + c, vec_M)

    if len(solution) == 4:
        est_M_np = matrix2numpy(M.subs(solution)).astype(float)
        frob = np.linalg.norm(est_M_np - true_M_np)

        estimate_2_0[seed - offset, 0] = est_M_np.round(3)
        frobnorm_2_0[seed - offset, 0] = frob
    else:
        frobnorm_2_0[seed - offset, 0] = -1
    '''


'\nindex_deletion = 0\noffset = 1\n\nc = np.hstack((vech_C, - b * unit_v))\nc = np.delete(c, (index_deletion))\n\nestimate_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)\nfrobnorm_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)\n\nfor seed in range(offset, 4 + offset):\n    true_M = simulate_M(seed=seed, p=p)\n\n    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)\n    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)\n\n    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)\n    emp_cov = np.cov(obs, rowvar=False)\n    emp_mean = np.mean(obs, axis=0)\n\n    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)\n    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)\n\n    # Linear regression\n    lin_reg = linear_model.LinearRegression()\n    lin_reg.fit(X=emp_A, y=-c)\n    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))\n    frob = np.linalg.norm(est_M - true_M)\n\n    estimate

In [6]:
index_deletion = 0
offset = 1

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_0[seed-offset, 0] = est_M.round(3)
    frobnorm_2_0[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_0[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_0[seed-offset, index+1] = frob

In [7]:
np.mean(frobnorm_2_0)

In [8]:
np.mean(frobnorm_2_0, axis=0)

array([np.float64(2.7157554964053765), np.float64(2.545726509136398),
       np.float64(1.938569857787821), np.float64(1.7151214860696669),
       np.float64(2.640028733500743)], dtype=object)

### $\texttt{index\_deletion = 1}$
***Results:*** 
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{2.092}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{2.119}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{2.088}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.93}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.773}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.552}$.

In [9]:
index_deletion = 1
offset = 12

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_1 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_1 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_1[seed-offset, 0] = est_M.round(3)
    frobnorm_2_1[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_1[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_1[seed-offset, index+1] = frob

In [10]:
np.mean(frobnorm_2_1)

In [11]:
np.mean(frobnorm_2_1, axis=0)

array([np.float64(2.119315702653408), np.float64(2.088465083478025),
       np.float64(1.9301759193918808), np.float64(1.7726052690995346),
       np.float64(2.5521114656440367)], dtype=object)

### $\texttt{index\_deletion = 2}$
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{2.515}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{2.794}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{2.721}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{2.315}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{2.155}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.59}$.

In [12]:
index_deletion = 2
offset = 123

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_2 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_2 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_2[seed-offset, 0] = est_M.round(3)
    frobnorm_2_2[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_2[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_2[seed-offset, index+1] = frob

In [13]:
np.mean(frobnorm_2_2)

In [14]:
np.mean(frobnorm_2_2, axis=0)

array([np.float64(2.794101922610868), np.float64(2.721633577749591),
       np.float64(2.3150668463328654), np.float64(2.1551806957225725),
       np.float64(2.5899355751988797)], dtype=object)

### $\texttt{index\_deletion = 3}$
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{2.587}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{2.824}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{2.93}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{2.35}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{2.114}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.715}$.

In [15]:
index_deletion = 3
offset = 1234

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_3 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_3 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_3[seed-offset, 0] = est_M.round(3)
    frobnorm_2_3[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_3[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_3[seed-offset, index+1] = frob

In [16]:
np.mean(frobnorm_2_3)

In [17]:
np.mean(frobnorm_2_3, axis=0)

array([np.float64(2.824341154172404), np.float64(2.9303714603360413),
       np.float64(2.34978225073085), np.float64(2.114454447367889),
       np.float64(2.7152310480809967)], dtype=object)

### $\texttt{index\_deletion = 4}$
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{2.007}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{1.984}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{2.201}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{2.017}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.623}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.562}$.

In [18]:
index_deletion = 4
offset = 12345

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_4 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_4 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_4[seed-offset, 0] = est_M.round(3)
    frobnorm_2_4[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_4[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_4[seed-offset, index+1] = frob

In [19]:
np.mean(frobnorm_2_4)

In [20]:
np.mean(frobnorm_2_4, axis=0)

array([np.float64(1.9836201825759558), np.float64(2.2010237972738462),
       np.float64(2.017090856341054), np.float64(1.622632204376296),
       np.float64(2.5622355548307683)], dtype=object)

### No deletion
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{12.004}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{50.428}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{3.422}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.891}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.663}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.615}$.

In [21]:
offset = 12345

c = np.hstack((vech_C, - b * unit_v))

estimate_2 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2[seed-offset, 0] = est_M.round(3)
    frobnorm_2[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2[seed-offset, index+1] = est_M.round(3)
        frobnorm_2[seed-offset, index+1] = frob

In [22]:
np.mean(frobnorm_2)

In [23]:
np.mean(frobnorm_2, axis=0)

array([np.float64(50.42792522643579), np.float64(3.4223765021488117),
       np.float64(1.8907988312286732), np.float64(1.6634908101760908),
       np.float64(2.615118831635098)], dtype=object)

### Overview: 2 nodes
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{3.931}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{10.478}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{2.652}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{2.074}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.841}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.612}$.

In [40]:
frobnorm_2nodes = np.vstack((frobnorm_2, frobnorm_2_0, frobnorm_2_1, frobnorm_2_2, frobnorm_2_3, frobnorm_2_4))
np.mean(frobnorm_2nodes, axis=0)

array([np.float64(10.477509947475678), np.float64(2.6515994883537877),
       np.float64(2.0735807603021867), np.float64(1.8405808188020054),
       np.float64(2.612443534815083)], dtype=object)

## 3 nodes

In [24]:
p = 3

C = 2 * np.eye(p)
vech_C = np.array([C[i, j] for i in range(p) for j in range(i, p)])
unit_v = np.eye(p)[:, index_intervention]
c = np.hstack((vech_C, - b * unit_v))

reg_param = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
num_param = len(reg_param) + 1 # also linear regression

### All entries
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{42.498}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{271.087}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{6.678}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{3.889}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{3.074}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.614}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{4.179}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.5$ is $\large{4.964}$.

In [25]:
offset = 120

estimate_3_0 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_0 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_3_0[seed-offset, 0] = est_M.round(3)
    frobnorm_3_0[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_3_0[seed-offset, index+1] = est_M.round(3)
        frobnorm_3_0[seed-offset, index+1] = frob

In [26]:
np.mean(frobnorm_3_0)

In [27]:
np.mean(frobnorm_3_0, axis=0)

array([np.float64(271.0869505212456), np.float64(6.677960064013381),
       np.float64(3.8892021537873354), np.float64(3.0740630243039218),
       np.float64(3.614438931735723), np.float64(4.178673630140045),
       np.float64(4.964049781815212)], dtype=object)

### One random entry missing
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{26.269}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{157.323}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{6.909}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{3.935}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{3.096}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.586}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{4.118}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.5$ is $\large{4.914}$.

In [28]:
entries = [[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1]]
offset = 1208

estimate_3_1 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_1 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_3_1[seed-offset, 0] = est_M.round(3)
    frobnorm_3_1[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_3_1[seed-offset, index+1] = est_M.round(3)
        frobnorm_3_1[seed-offset, index+1] = frob

In [29]:
np.mean(frobnorm_3_1)

In [30]:
np.mean(frobnorm_3_1, axis=0)

array([np.float64(157.3228586630324), np.float64(6.90868449862348),
       np.float64(3.9345583586320756), np.float64(3.095995498317779),
       np.float64(3.585962697397941), np.float64(4.117869731235047),
       np.float64(4.914213703261751)], dtype=object)

### Two entries missing I
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{21.27}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{122.947}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{6.478}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{3.765}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{3.061}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.564}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{4.129}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.5$ is $\large{4.945}$.

In [31]:
offset = 12089

estimate_3_2 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_2 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_3_2[seed-offset, 0] = est_M.round(3)
    frobnorm_3_2[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_3_2[seed-offset, index+1] = est_M.round(3)
        frobnorm_3_2[seed-offset, index+1] = frob

In [32]:
np.mean(frobnorm_3_2)

In [33]:
np.mean(frobnorm_3_2, axis=0)

array([np.float64(122.94716426965168), np.float64(6.478183491982295),
       np.float64(3.765169291652454), np.float64(3.0605375312378844),
       np.float64(3.5639319978416886), np.float64(4.128631268683257),
       np.float64(4.945495720270547)], dtype=object)

### Two entries missing II
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{33.845}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{210.425}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{6.696}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{3.849}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{3.079}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.568}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{4.117}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.5$ is $\large{4.909}$.

In [34]:
offset = 120899

estimate_3_3 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_3 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_3_3[seed-offset, 0] = est_M.round(3)
    frobnorm_3_3[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_3_3[seed-offset, index+1] = est_M.round(3)
        frobnorm_3_3[seed-offset, index+1] = frob

In [35]:
np.mean(frobnorm_3_3)

In [36]:
np.mean(frobnorm_3_3, axis=0)

array([np.float64(210.4247612747219), np.float64(6.968713742087751),
       np.float64(3.849257508862546), np.float64(3.0786416291554604),
       np.float64(3.5676437666765057), np.float64(4.117166184808064),
       np.float64(4.908770473115364)], dtype=object)

### Overview: 3 nodes
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{30.97}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{190.445}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{6.758}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{3.86}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{3.077}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.583}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{4.136}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.5$ is $\large{4.933}$.

In [42]:
frobnorm_3nodes = np.vstack((frobnorm_3_0, frobnorm_3_1, frobnorm_3_2, frobnorm_3_3))
np.mean(frobnorm_3nodes, axis=0)

array([np.float64(190.4454336821627), np.float64(6.758385449176724),
       np.float64(3.8595468282335976), np.float64(3.0773094207537555),
       np.float64(3.5829943484129636), np.float64(4.135585203716608),
       np.float64(4.933132419615719)], dtype=object)